In [ ]:
# !pip install transformers

In [6]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F
def compute_s_score (article, candidate, tokenizer, model):

  article_embed = tokenizer (article, max_length = 512, padding = 'max_length', return_tensors = 'pt', truncation = True).to (device)
  candidate_embed = tokenizer (candidate, max_length = 512, padding = 'max_length', return_tensors = 'pt', truncation = True).to (device)

  h_d = model (article_embed['input_ids'])[1]   # [CLS] representations from pooling
  x_output = model (candidate_embed['input_ids'])
  h_x = x_output[1]
  
  # Raw [CLS] representation
  # h_d = model (article_embed['input_ids'])[:, 0,]    
  # h_x = model (candidate_embed['input_ids'])[:, 0,]
  
  sim = nn.CosineSimilarity(dim=1, eps=1e-6)
  s_score = sim (h_d, h_x)

  return s_score, x_output[0]


In [3]:
cad = "I like apple."
art = 'I like pears.'
sentences = [
    "Trump is a dump ass.",
    "Do you love fishing?",
    "Dinosaurs are exstinct thousands of years ago.",
    "Iowa is the best state.",
    "The state beside Utah is of the first rank.",
    "The giant retile died out a long time ago",
]

In [4]:
tokenizer = AutoTokenizer.from_pretrained ("bert-base-uncased")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoModel.from_pretrained ('bert-base-uncased').to (device)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
s_score, _ = compute_s_score (art, cad, tokenizer, model)  
batch_s_score, _ = compute_s_score (sentences, sentences, tokenizer, model)  

In [8]:
s_score, batch_s_score

(tensor([0.9994], device='cuda:0', grad_fn=<SumBackward1>),
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000], device='cuda:0',
        grad_fn=<SumBackward1>))